In [2]:
# Had issues installing pyreadstat so used magic command in notebook instead
%pip install pyreadstat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 560.9/560.9 kB 3.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
import glob
import pyreadstat
import pandas as pd
from datetime import datetime
from pathlib import Path

In [ ]:
#todo make cross-platform safe, or just add your won absolute path 
DATA_DIR = Path('/Volumes/WRKGRP/STD-FSW-BSI-SD-Movement_Tracking/dsp/data')

filename = 'tracking-data-info'
path = DATA_DIR / '02_interim' / f"{filename}.parquet"

if path.exists():
    print(f'Found {path}')
    tracking_data = pd.read_parquet(path)
    display(df)
else:
    # Get 2023 school/class info from tracking data subfolder names
    folder = DATA_DIR / '01_tracking' / '2023'
    subfolders = [f.name for f in folder.iterdir() if f.is_dir()]
    
    # Separate subfolder name into date, school, class
    data = []
    
    for subfolder in subfolders:
        date, school_num = subfolder.split('_s')
        school_num, class_num = school_num.split('_c')
        data.append([date, school_num, class_num])
    
    # Create df of date, school, class
    tracking_data = pd.DataFrame(data, columns=['date', 'school', 'class']).astype(str)
    tracking_data['date'] = pd.to_datetime(tracking_data['date'], format='%Y_%m_%d')
    
    # Save df as parquet file to retain dtypes
    path.parent.mkdir(parents=True, exist_ok=True)
    tracking_data.to_parquet(path, index=False)
    print("Data saved to", path)

In [43]:
tracking_data

,date,school,class
0,2023-04-11,42,102
1,2023-05-11,43,103
2,2023-05-23,46,107
3,2023-05-24,47,108
4,2023-05-31,1,104
5,2023-06-08,45,105
6,2023-06-09,45,106


Teacher survey responses look like they were collected through an online form. Delivered raw as wide-format SPSS files. 

# IOP scores

Confirmed with Nathalie that IOP responses were optional.
If IOP response was given, variable name contains 'Q68' and student number. It's then followed by Q70 and Q71 with matching student number.

School and class data has been added to each dataframe from source filename. This data is surely already present in the questionnaire, but I cannot decipher under which variable it's present. Hence, a workaround.

In [59]:
# Determine path to raw SPSS files
folder = DATA_DIR / '01_survey' / 'teacher_raw_2023'

# Initiate empty dict to store teacher questionnaire dfs
tq_all = {}

for file in folder.glob('*.sav'):
    var_name = file.stem
    df = pd.read_spss(file)
    
    # Add school/class as columns to each df from filename
    school_num, class_num = var_name.split('_')[1:]
    df['school'] = school_num
    df['class'] = class_num
    
    # Store df in dict with filename as key
    tq_all[var_name] = df
    print(f'Loaded dataframe: {var_name}')
    
print(f'Total dataframes: {len(tq_all)}')

Loaded dataframe: tq_49_113
Loaded dataframe: tq_1_104
Loaded dataframe: tq_41_100
Loaded dataframe: tq_41_101
Loaded dataframe: tq_42_102
Loaded dataframe: tq_43_103
Loaded dataframe: tq_45_105
Loaded dataframe: tq_45_106
Loaded dataframe: tq_46_107
Loaded dataframe: tq_47_108
Loaded dataframe: tq_49_110
Loaded dataframe: tq_49_111
Loaded dataframe: tq_49_112
Total dataframes: 13


In [60]:
# Inspect available columns
print(tq_all['tq_1_104'].columns.tolist())

['StartDate', 'EndDate', 'Status', 'Progress', 'Duration__in_seconds_', 'Finished', 'RecordedDate', 'ResponseId', 'DistributionChannel', 'UserLanguage', 'Q8', 'Q30', 'Q31', 'Q32', 'Q27', 'Q28', 'Q29', 'Q29_6_TEXT', 'Q30.0', 'Q31.0', 'Q32.0', 'Q33', 'Q33_2_TEXT', 'Q34_1', 'Q34_2', 'Q34_3', 'Q34_4', 'Q34_5', 'Q34_6', 'Q14_1', 'Q14_2', 'Q14_3', 'Q14_4', 'Q14_5', 'Q14_6', 'Q14_7', 'Q14_8', 'Q14_9', 'Q14_10', 'Q14_11', 'Q14_12', 'Q14_13', 'Q14_14', 'Q14_15', 'Q14_16', 'Q15_1', 'Q15_2', 'Q15_3', 'Q15_4', 'Q15_5', 'Q15_6', 'Q15_7', 'Q15_8', 'Q15_9', 'Q15_10', 'Q15_11', 'Q15_12', 'Q15_13', 'Q15_14', 'Q15_15', 'Q15_16', 'A1_Q60_1', 'A1_Q60_2', 'A1_Q60_3', 'A1_Q60_4', 'A1_Q60_5', 'A1_Q60_6', 'A1_Q60_7', 'A1_Q60_8', 'A1_Q60_9', 'A1_Q60_10', 'A1_Q60_11', 'A1_Q60_12', 'A1_Q60_13', 'A1_Q60_14', 'A1_Q60_15', 'A1_Q60_16', 'A1_Q60_17', 'A2_Q60_1', 'A2_Q60_2', 'A2_Q60_3', 'A2_Q60_4', 'A2_Q60_5', 'A2_Q60_6', 'A2_Q60_7', 'A2_Q60_8', 'A2_Q60_9', 'A2_Q60_10', 'A2_Q60_11', 'A2_Q60_12', 'A2_Q60_13', 'A2_Q60_1

In [62]:
# Initiate empty dict to store relevant tq only
tq_relevant = {}

for df in tq_all:
    school_num = str(tq_all[df]['school'].iloc[0])
    class_num = str(tq_all[df]['class'].iloc[0])
    
    tq_match = tracking_data[
        (tracking_data['school'] == school_num) &
        (tracking_data['class'] == class_num)
    ]
    
    if not tq_match.empty:
        tq_relevant[df] = tq_all[df]
        print(f"Matching dataframe: {df}")

#todo concatenate AFTER the columns have been equalized
# tq = pd.concat(tq_relevant, ignore_index=True)

print(f"Total matching: {len(tq_relevant)}")

Matching dataframe: tq_1_104
Matching dataframe: tq_42_102
Matching dataframe: tq_43_103
Matching dataframe: tq_45_105
Matching dataframe: tq_45_106
Matching dataframe: tq_46_107
Matching dataframe: tq_47_108
Total matching: 7


Based on reading the codebook and inspecting the answers in the raw files, I've determined the following:
* Q27 = T_gender
* Q28 = T_age
* Q29 = T_dutch
* Q30 = T_exp1
* Q31 = T_exp2
* Q32 = T_time_teaching
* Q33 = T_class_comp

In [ ]:
tq_relevant['tq_1_104']

In [65]:
# Eliminating irrelevant columns in tq dataframes

# Lists of relevant questions
descriptives = ['Q27', 'Q28', 'Q29', 'Q30', 'Q31', 'Q32', 'Q33']
iop_id = ['Q68', 'Q70', 'Q71']

tq_filtered = {}

for key, df in tq_relevant.items():
    # Create a mask for columns to keep
    cols_to_keep = []
    
    for col in df.columns:
        # Check if column matches any descriptive column
        if any(q_id in col for q_id in descriptives):
            cols_to_keep.append(col)
        # Check if column contains any of the specified question IDs
        elif any(q_id in col for q_id in iop_id):
            cols_to_keep.append(col)
    
    # Create a new dataframe with only the columns to keep
    tq_filtered[key] = df[cols_to_keep]

In [ ]:
# Check long-format 
tq_filtered['tq_1_104'].melt()

# Load keyfiles

In [ ]:
# Load keyfile tag number-Track Lab ID
path = DATA_DIR / 'keyfiles' / 'Keyfile_csv.csv'

keyfile_tagID = pd.read_csv(path, delimiter=';')

In [92]:
folder = DATA_DIR / 'keyfiles'
# file_path = folder.glob('*.xlsx')

keyfiles = pd.DataFrame()

for file in folder.glob('*.xlsx'):
    try:
        df = pd.read_excel(file, engine='openpyxl')
        keyfiles = pd.concat([keyfiles, df], ignore_index=True)
        print(f'Loaded: {file.name}')
    except Exception as e:
        print(f'Could not read {file.name}: {e}')
        df = pd.read_csv(file)
        keyfiles = pd.concat([keyfiles, df], ignore_index=True)
        print('it worked')

Loaded: keyfile school 47 class 108.xlsx
Loaded: keyfile school 1 class 104.xlsx
Loaded: keyfile school 41 class 100.xlsx
Loaded: keyfile school 41 class 101.xlsx
Loaded: keyfile school 42 class 102.xlsx
Loaded: keyfile school 43 class 103.xlsx
Loaded: keyfile school 45 class 105.xlsx
Loaded: keyfile school 45 class 106.xlsx
Loaded: keyfile school 46 class 107.xlsx
Could not read ~$keyfile school 43 class 103.xlsx: File is not a zip file
it worked


In [93]:
keyfiles

,school ID,klas ID,subject ID,id,voornaam,achternaam,consent,tagnummer,tagnummer,tagnr.,Unnamed: 8,tagnr,trackingnnumer,Unnamed: 9,Unnamed: 10,Oliver Irvine
0,47.0,108.0,1.0,2223.0,NaN,NaN,1.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,47.0,108.0,2.0,2224.0,NaN,NaN,1.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,47.0,108.0,3.0,2225.0,NaN,NaN,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,47.0,108.0,4.0,2226.0,NaN,NaN,1.0,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,47.0,108.0,5.0,2227.0,NaN,NaN,1.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,46,107.0,17.0,2219.0,NaN,NaN,1,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN
205,46,107.0,18.0,2220.0,NaN,NaN,1,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN
206,46,107.0,19.0,2221.0,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,46,107.0,20.0,2222.0,NaN,NaN,received oral permission from parent on day of...,NaN,NaN,9.0,"since student had no written consent, their na...",NaN,NaN,NaN,NaN,NaN
